# target encoder & ohe & FM

In [1]:
%load_ext autoreload
%autoreload 2

import os
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append(os.path.abspath('..'))
# ---------------------------------
import numpy as np
import pandas as pd
import scipy
import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.metrics import roc_auc_score

from hyperopt import hp
# ---------------------------------
from tools import CV, Tuning, CVGetScore, IdxValEncoder, fm

In [2]:
train_df = pd.read_csv('../data/train.csv', index_col='id')
test_df = pd.read_csv('../data/test.csv', index_col='id')

# ord_5
for i in range(2):
    train_df[f'ord_5_{i}'] = train_df['ord_5'].str[i]
    test_df[f'ord_5_{i}'] = test_df['ord_5'].str[i]

# fillna
for col in test_df.columns:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)
    test_df[col].fillna(test_df[col].mode()[0], inplace=True)

# target
target = train_df['target']
y_train = target.values

# drop
train_df.drop(['target', 'ord_5'], axis=1, inplace=True)
test_df.drop(['ord_5'], axis=1, inplace=True)

In [3]:
feature_col = train_df.columns

bin_col = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']

class_col = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4',
             'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9',
             'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4',
             'day', 'month', 'ord_5_0', 'ord_5_1']

In [4]:
for col in bin_col:
    map_dict = dict(zip(train_df[col].unique(), [0., 1.]))
    train_df[col] = train_df[col].map(map_dict)
    test_df[col] = test_df[col].map(map_dict)

In [5]:
ecd = IdxValEncoder(feature_col, bin_col, class_col)
ecd.fit(train_df, verbose=1)

600000it [00:11, 52322.58it/s]


In [6]:
idx, val = ecd.transform(train_df, verbose=1)

600000it [00:12, 48506.24it/s]


# fit one

In [7]:
# fit param
batch_size = 8192

callback = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', patience=2, mode='max')
fit_param = {'batch_size': batch_size,
             'epochs':10,
             'verbose': 1, 
             'callbacks':[callback]}

# model_param
opt = tf.keras.optimizers.Adam(learning_rate=5e-4)

model_param = {'vocabulary_size':ecd.get_vocabulary(), 
               'feature_number': len(feature_col),
               'activation': 'sigmoid',
               'metrics': ['AUC'],
               'optimizer': opt, 
               'k': 5, 'loss': tf.keras.losses.BinaryCrossentropy()}

In [8]:
model = fm(**model_param)
cv = CV(model, 5)

In [9]:
cv.fit(x=[idx, val],
       y=y_train, 
       metrics_func=roc_auc_score,
       split_method=StratifiedKFold,
       fit_params=fit_param,
       eval_param={'batch_size': batch_size},
       use_proba=False, 
       verbose=True,
       fit_use_valid=True)

Train on 480000 samples, validate on 120000 samples
Epoch 1/10
480000/480000 [==============================] - 3s 6us/sample - loss: 0.6052 - AUC: 0.5123 - val_loss: 0.5464 - val_AUC: 0.5281
Epoch 2/10
480000/480000 [==============================] - 2s 3us/sample - loss: 0.5166 - AUC: 0.5304 - val_loss: 0.4945 - val_AUC: 0.5681
Epoch 3/10
480000/480000 [==============================] - 2s 3us/sample - loss: 0.4820 - AUC: 0.6252 - val_loss: 0.4705 - val_AUC: 0.6982
Epoch 4/10
480000/480000 [==============================] - 2s 3us/sample - loss: 0.4557 - AUC: 0.7430 - val_loss: 0.4431 - val_AUC: 0.7570
Epoch 5/10
480000/480000 [==============================] - 2s 4us/sample - loss: 0.4298 - AUC: 0.7699 - val_loss: 0.4238 - val_AUC: 0.7700
Epoch 6/10
480000/480000 [==============================] - 2s 3us/sample - loss: 0.4141 - AUC: 0.7812 - val_loss: 0.4140 - val_AUC: 0.7766
Epoch 7/10
480000/480000 [==============================] - 2s 4us/sample - loss: 0.4058 - AUC: 0.7875 - val

0.7826533289826314

In [11]:
# fit param
batch_size = 8192

callback = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', patience=2, mode='max')
fit_param = {'batch_size': batch_size,
             'epochs':200,
             'verbose': 1, 
             'callbacks':[callback]}

# model_param
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

model = cv.model
for i in model:
    i.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.BinaryCrossentropy(), metrics=['AUC'])
    
cv = CV(model, 5)

cv.fit(x=[idx, val],
       y=y_train, 
       metrics_func=roc_auc_score,
       split_method=StratifiedKFold,
       fit_params=fit_param,
       eval_param={'batch_size': batch_size},
       use_proba=False, 
       verbose=True,
       fit_use_valid=True)

Train on 480000 samples, validate on 120000 samples
Epoch 1/200
480000/480000 [==============================] - 3s 6us/sample - loss: 0.3950 - AUC: 0.7960 - val_loss: 0.4044 - val_AUC: 0.7828
Epoch 2/200
480000/480000 [==============================] - 2s 4us/sample - loss: 0.3950 - AUC: 0.7960 - val_loss: 0.4044 - val_AUC: 0.7828
Epoch 3/200
480000/480000 [==============================] - 2s 4us/sample - loss: 0.3949 - AUC: 0.7960 - val_loss: 0.4043 - val_AUC: 0.7828
Epoch 4/200
480000/480000 [==============================] - 2s 4us/sample - loss: 0.3949 - AUC: 0.7960 - val_loss: 0.4043 - val_AUC: 0.7828
Epoch 5/200
480000/480000 [==============================] - 2s 4us/sample - loss: 0.3948 - AUC: 0.7961 - val_loss: 0.4043 - val_AUC: 0.7829
Epoch 6/200
480000/480000 [==============================] - 2s 4us/sample - loss: 0.3948 - AUC: 0.7961 - val_loss: 0.4043 - val_AUC: 0.7829
Epoch 7/200
480000/480000 [==============================] - 2s 4us/sample - loss: 0.3947 - AUC: 0.796

0.7826850592374202

# Tuning

In [ ]:
# fit param
callback = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', patience=2, mode='max')
fit_param = {'batch_size': 4096, 'epochs':200, 'verbose': 0, 'callbacks':[callback]}

cv_fit_param = {'fit_params': fit_param, 
                'eval_param': {'batch_size':4096},
                'use_proba':False, 
                'fit_use_valid': True}

# model_fix_param & model_search_space
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

model_fix_param = {'vocabulary_size':ecd.get_vocabulary(), 
                   'feature_number': len(feature_col),
                   'activation': 'sigmoid',
                   'metrics': ['AUC'],
                   'optimizer': opt, 
                   'k': 10}

model_search_space = {'loss': hp.choice('loss', ['MSE', tf.keras.losses.BinaryCrossentropy()]),
                      'l1': hp.loguniform('l1', -10, 0),
                      'l2': hp.loguniform('l2', -10, 0)}

# cv get score
def neg_auc(y_true, y_pred):
    return - roc_auc_score(y_true, y_pred)

gs = CVGetScore(x=[idx, val],
                y=y_train, 
                metrics_func=neg_auc,
                split_method=StratifiedKFold,
                nfolds=5, 
                random_state=2333,
                model=fm, 
                cv_fit_params=cv_fit_param, 
                model_fix_params=model_fix_param, 
                model_search_space=model_search_space)

tuning = Tuning(gs, verbose=1)
tuning.fmin(gs.GET_SEARCH_SPACE(), max_evals=100)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

model = linear_regression(vocabulary_size=ecd.get_vocabulary(), 
                          feature_number=len(feature_col),, 
                          activation='sigmoid',
                          loss='mse',
                          metrics=['AUC'],
                          optimizer=opt, 
                          l1=0., l2=0.)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', patience=5, mode='max')
fit_param = {'batch_size':1024, 'epochs':20, 'verbose': 1, 'callbacks':[callback]}

In [ ]:
pred_param = {'batch_size':1024}

cv = CV(model, 5)

cv.fit(x=[idx, val],
       y=y_train, 
       metrics_func=roc_auc_score,
       split_method=StratifiedKFold,
       fit_params=fit_param,
       eval_param=pred_param,
       use_proba=False, 
       verbose=True,
       fit_use_valid=True)

In [ ]:
np.exp(-10)